# **Challenge Kaggle - Análise Preditiva do Titanic**
**Autor:** Luiz Marques

> Objetivo: Este notebook explora o famoso dataset do Titanic do Kaggle. O objetivo é construir um modelo de machine learning capaz de prever se um passageiro sobreviveu ou não ao desastre, com base em suas características.

## **1. Configuração Inicial e Análise Exploratória (EDA)**
> A primeira etapa de qualquer projeto de ciência de dados é carregar os dados e realizar uma análise exploratória para entender sua estrutura, identificar desafios como dados faltantes e começar a formular hipóteses.

## **1.1. Carregando os Dados**

In [1]:
# Baixando dados do drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Caminho para o arquivo dentro do seu Google Drive
caminho_arquivo_treino = '/content/drive/MyDrive/Kaggle_Titanic/train.csv'

# Ler o arquivo e carregá-lo em um DataFrame
df_treino = pd.read_csv(caminho_arquivo_treino)

# Visualizar as 5 primeiras linhas para confirmar que tudo deu certo
df_treino.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Caminho para o arquivo dentro do seu Google Drive
caminho_arquivo_teste = '/content/drive/MyDrive/Kaggle_Titanic/test.csv'

# Ler o arquivo e carregá-lo em um DataFrame
df_teste = pd.read_csv(caminho_arquivo_teste)

# Visualizar as 5 primeiras linhas para confirmar que tudo deu certo
df_teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## **1.2. Investigando a Estrutura dos Dados**

In [4]:
# Visualizando dados com o info
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


- Primeiras Descobertas:

1. A coluna Age tem 177 valores faltantes.

2. A coluna Cabin tem 687 valores faltantes (a grande maioria).

3. A coluna Embarked tem 2 valores faltantes.

4. Colunas como Name, Sex, Ticket, Cabin e Embarked são do tipo object e precisarão ser transformadas em números para os modelos.

In [5]:
# Visualizando quantidade de linhas e colunas
df_treino.shape

(891, 12)

In [6]:
# Visualizando das descrições
df_treino.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Insights Iniciais:

1. Aproximadamente 38% dos passageiros no conjunto de treino sobreviveram.

2. A idade média dos passageiros é de aproximadamente 30 anos.

3. A maioria dos passageiros viajava sem irmãos/cônjuges (SibSp) ou pais/filhos (Parch).

# **2. Limpeza e Pré-processamento Inicial**
> Com base nos insights da EDA, a próxima etapa é preparar os dados para a modelagem. Isso envolve tratar os valores ausentes e transformar os dados para um formato numérico.

In [7]:
# Calcular a mediana da idade APENAS no conjunto de treino
mediana_idade = df_treino['Age'].median()
print(f"A mediana da idade é: {mediana_idade}")

# Agora, vamos preencher os valores nulos (NaN) com essa mediana nos dois dataframes
df_treino['Age'].fillna(mediana_idade, inplace=True)
df_teste['Age'].fillna(mediana_idade, inplace=True) # Usando a mesma mediana do treino

A mediana da idade é: 28.0


/tmp/ipython-input-2382831653.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_treino['Age'].fillna(mediana_idade, inplace=True)
/tmp/ipython-input-2382831653.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [8]:
# Encontrar o porto de embarque mais comum (a moda)
moda_embarked = df_treino['Embarked'].mode()[0]
print(f"O porto mais comum é: {moda_embarked}")

# Preencher os valores faltantes com a moda
df_treino['Embarked'].fillna(moda_embarked, inplace=True)

O porto mais comum é: S


/tmp/ipython-input-3987722532.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_treino['Embarked'].fillna(moda_embarked, inplace=True)


In [9]:
# Remover a coluna 'Cabin' de ambos os dataframes
df_treino = df_treino.drop('Cabin', axis=1)
df_teste = df_teste.drop('Cabin', axis=1)

In [10]:
# Preenchendo o valor faltante em Fare no conjunto de teste
mediana_fare = df_treino['Fare'].median()
df_teste['Fare'].fillna(mediana_fare, inplace=True)

# Verificação final de valores nulos
print("Dados faltantes no TREINO após tratamento:")
print(df_treino.isnull().sum())

print("\nDados faltantes no TESTE após tratamento:")
print(df_teste.isnull().sum())

Dados faltantes no TREINO após tratamento:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

Dados faltantes no TESTE após tratamento:
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


/tmp/ipython-input-3191006267.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_teste['Fare'].fillna(mediana_fare, inplace=True)


## **2.2. Transformação de Features Categóricas**
- Os modelos de machine learning requerem input numérico. Vamos converter as colunas de texto.



In [11]:
# Criar um dicionário para o mapeamento: male -> 0, female -> 1
sex_map = {'male': 0, 'female': 1}

# Aplicar o mapeamento nos dois dataframes
df_treino['Sex'] = df_treino['Sex'].map(sex_map)
df_teste['Sex'] = df_teste['Sex'].map(sex_map)

### Temos três portos de embarque: 'S', 'C' e 'Q'. Não podemos simplesmente mapeá-los para 0, 1 e 2, pois isso criaria uma relação de ordem que não existe (o modelo poderia pensar que 2 > 1 > 0).

- A técnica correta aqui é o One-Hot Encoding. Ela cria uma nova coluna para cada categoria. Por exemplo, teremos uma coluna Embarked_S que será 1 se o passageiro embarcou em Southampton e 0 caso contrário. O pandas faz isso facilmente.

In [12]:
# Aplicar One-Hot Encoding
# O drop_first=True remove uma das categorias para evitar redundância de dados

df_treino = pd.get_dummies(df_treino, columns=['Embarked'], drop_first=True, dtype=int)
df_teste = pd.get_dummies(df_teste, columns=['Embarked'], drop_first=True, dtype=int)

## Removendo Colunas Irrelevantes (Por enquanto)

As colunas Name e Ticket são únicas para quase todos os passageiros. Embora seja possível extrair informações úteis delas (como o título "Mr.", "Miss.", etc., do nome), para um primeiro modelo, elas geralmente adicionam mais ruído do que sinal. A abordagem mais simples e eficaz agora é removê-las.

In [13]:
# Remover as colunas 'Name' e 'Ticket' de ambos os dataframes
df_treino = df_treino.drop(['Name', 'Ticket'], axis=1)
df_teste = df_teste.drop(['Name', 'Ticket'], axis=1)

In [14]:
# Visualizar as primeiras linhas do dataframe de treino transformado
df_treino.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,1,0,3,0,22.0,1,0,7.2500,0,1
1,2,1,1,1,38.0,1,0,71.2833,0,0
2,3,1,3,1,26.0,0,0,7.9250,0,1
3,4,1,1,1,35.0,1,0,53.1000,0,1
4,5,0,3,0,35.0,0,0,8.0500,0,1


In [15]:
# Verificar os tipos de dados de todas as colunas
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked_Q   891 non-null    int64  
 9   Embarked_S   891 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 69.7 KB


## **3. Modelagem e Primeira Submissão**
> Com um dataset limpo, vamos treinar nosso primeiro modelo para estabelecer uma baseline de performance.

### **3.1. Avaliação Local e Competição de Modelos**
- Antes de submeter, testamos vários algoritmos em um conjunto de validação local para identificar o candidato mais promissor.

In [16]:
# A coluna 'PassengerId' é apenas um identificador, não ajuda na previsão, então removemos.
# A coluna 'Survived' é nosso alvo, então ela não pode estar nas features X.
X = df_treino.drop(['Survived', 'PassengerId'], axis=1)
y = df_treino['Survived']

# Vamos dar uma olhada rápida para confirmar
print("Formato de X:", X.shape)
print("Formato de y:", y.shape)

Formato de X: (891, 8)
Formato de y: (891,)


## Criando Conjuntos de Treino e Validação

- Agora, vamos pegar o X e y e dividi-los para que tenhamos um conjunto para treinar e outro para validar (nosso "simulado"). Usaremos 80% dos dados para treinar e 20% para validar.

In [17]:
# Import do train_test_split
from sklearn.model_selection import train_test_split

# O random_state=42 garante que a divisão seja sempre a mesma, tornando nosso experimento reproduzível.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Escolhendo e Treinando o Modelo

- Para começar, vamos usar um dos modelos mais clássicos e eficientes para problemas de classificação como este: a Regressão Logística.

In [18]:
# Import Regressão Logística
from sklearn.linear_model import LogisticRegression

# 1. Criar uma instância do modelo
# max_iter=200 ajuda a garantir que o modelo encontre a melhor solução
modelo = LogisticRegression(max_iter=200)

# 2. Treinar o modelo com nossos dados de TREINO. Este é o momento em que o modelo "aprende".
modelo.fit(X_train, y_train)


LogisticRegression(max_iter=200)

## Avaliando a Performance do Modelo
- Analisando como o modelo se sai prevendo os resultados do nosso conjunto de validação (X_val).
- A acurácia nos diz a porcentagem de passageiros que o modelo classificou corretamente (acertou se sobreviveu ou não).

In [19]:
# Import acurácia
from sklearn.metrics import accuracy_score

# 1. Fazer previsões no conjunto de validação
previsoes = modelo.predict(X_val)

# 2. Comparar as previsões do modelo com as respostas reais (y_val)
acuracia = accuracy_score(y_val, previsoes)

print(f"A acurácia do nosso modelo no conjunto de validação é: {acuracia * 100:.2f}%")

A acurácia do nosso modelo no conjunto de validação é: 81.01%


## Comparando Múltiplos Modelos
* Vamos treinar e avaliar 3 novos candidatos, além da nossa Regressão Logística, para ver qual se sai melhor. Nossos competidores serão:

1. Regressão Logística: Nosso baseline.

2. Árvore de Decisão: Um modelo que toma decisões baseadas em regras (ex: "se Sex é feminino, vá para este galho...").

3. Random Forest: Um modelo muito poderoso que é basicamente um comitê de muitas Árvores de Decisão. É um dos modelos mais populares e eficazes para problemas como este.

4. Support Vector Machine (SVM): Outro tipo de classificador poderoso que tenta encontrar a melhor "linha" ou "plano" para separar as classes.

**O processo será o mesmo para todos: treinar com (X_train, y_train) e avaliar com (X_val, y_val). Usar os mesmos dados de treino e validação para todos garante uma comparação justa.**

In [20]:
# 1. Importar todos os modelos que vamos testar
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC # "C" de Classifier
from sklearn.metrics import accuracy_score

# 2. Criar um dicionário com as instâncias dos modelos
# Usamos random_state=42 para garantir que os resultados dos modelos baseados em aleatoriedade sejam reprodutíveis
modelos = {
    "Regressão Logística": LogisticRegression(max_iter=200),
    "Árvore de Decisão": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC()
}

# 3. Dicionário para armazenar os resultados
resultados_acuracia = {}

# 4. Loop para treinar e avaliar cada modelo
print("Iniciando a competição de modelos...")
for nome, modelo in modelos.items():
    # Treinar o modelo
    modelo.fit(X_train, y_train)

    # Fazer previsões no conjunto de validação
    previsoes_val = modelo.predict(X_val)

    # Calcular e armazenar a acurácia
    acuracia = accuracy_score(y_val, previsoes_val)
    resultados_acuracia[nome] = acuracia

    # Imprimir o resultado
    print(f"  - Acurácia do modelo '{nome}': {acuracia * 100:.2f}%")

# 5. Encontrar e anunciar o vencedor
vencedor_nome = max(resultados_acuracia, key=resultados_acuracia.get)
vencedor_acuracia = resultados_acuracia[vencedor_nome]

print(f"\n🏆 O modelo vencedor foi '{vencedor_nome}' com uma acurácia de {vencedor_acuracia * 100:.2f}%.")

Iniciando a competição de modelos...
  - Acurácia do modelo 'Regressão Logística': 81.01%
  - Acurácia do modelo 'Árvore de Decisão': 78.77%
  - Acurácia do modelo 'Random Forest': 79.89%
  - Acurácia do modelo 'SVM': 65.36%

🏆 O modelo vencedor foi 'Regressão Logística' com uma acurácia de 81.01%.


## **3.2. Gerando a Primeira Submissão**
- Com base na avaliação local, a Regressão Logística foi escolhida. Treinamos o modelo com todos os dados de treino e geramos o arquivo de submissão.

## **Versão 1 (Logistic Regression)**

In [21]:
# Lembrando que 'X' e 'y' são nossos dataframes de treino completos
# X = df_treino.drop(['Survived', 'PassengerId'], axis=1)
# y = df_treino['Survived']
# E 'df_teste' é o nosso dataframe de teste já limpo e transformado.

# 1. Treinar o modelo final com TODOS os dados de treino
print("Treinando o modelo final com todos os dados de treino...")
modelo_final = LogisticRegression(max_iter=200)
modelo_final.fit(X, y)
print("Modelo final treinado com sucesso!")

# 2. Preparar os dados de teste finais
# Garantimos que as colunas de teste sejam exatamente as mesmas que as de treino
X_teste_final = df_teste[X.columns]

# 3. Fazer as previsões no conjunto de teste do Kaggle
print("Fazendo as previsões finais...")
previsoes_finais = modelo_final.predict(X_teste_final)

# 4. Criar o dataframe de submissão no formato exigido pelo Kaggle
# Duas colunas: 'PassengerId' e 'Survived'
submissao = pd.DataFrame({
    "PassengerId": df_teste["PassengerId"],
    "Survived": previsoes_finais
})

# 5. Salvar o dataframe em um arquivo .csv
# O 'index=False' é MUITO importante para não criar uma coluna extra de índice no arquivo
submissao.to_csv('minha_submissao.csv', index=False)

print("\nArquivo 'minha_submissao.csv' foi criado!")
print("Ele está pronto para ser enviado para a competição no Kaggle.")

# Visualizar as 5 primeiras linhas do arquivo que acabamos de criar
submissao.head()

Treinando o modelo final com todos os dados de treino...
Modelo final treinado com sucesso!
Fazendo as previsões finais...

Arquivo 'minha_submissao.csv' foi criado!
Ele está pronto para ser enviado para a competição no Kaggle.


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


# **Versão 2: Melhorando a Pontuação com Random Forest**

Na nossa primeira submissão, utilizamos um modelo de Regressão Logística. Após todo o tratamento dos dados, alcançamos uma pontuação pública no Kaggle de **0.76794**.

Este é um resultado sólido que superou as baselines mais simples. No entanto, em nossa análise comparativa de modelos, o `Random Forest` também se mostrou um forte candidato. Nesta seção, vamos treiná-lo com todos os dados e gerar uma nova submissão para verificar se sua maior complexidade pode capturar mais padrões nos dados e, consequentemente, melhorar nossa pontuação.

In [22]:
# 1. Criar e treinar o modelo Random Forest com TODOS os dados de treino
print("Treinando o modelo Random Forest com todos os dados de treino...")
# n_estimators=100 é um bom ponto de partida. random_state=42 garante a reprodutibilidade.
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X, y)
print("Modelo Random Forest treinado com sucesso!")

# 2. Preparar os dados de teste finais (garantindo que as colunas são as mesmas)
X_teste_final = df_teste[X.columns]

# 3. Fazer as previsões no conjunto de teste do Kaggle
print("Fazendo as previsões com o Random Forest...")
previsoes_rf = modelo_rf.predict(X_teste_final)

# 4. Criar o dataframe de submissão
submissao_rf = pd.DataFrame({
    "PassengerId": df_teste["PassengerId"],
    "Survived": previsoes_rf
})

# 5. Salvar em um NOVO arquivo .csv
# É MUITO IMPORTANTE usar um nome de arquivo diferente!
submissao_rf.to_csv('submissao_random_forest.csv', index=False)

print("\nArquivo 'submissao_random_forest.csv' foi criado com sucesso!")
print("Ele está pronto para uma nova submissão no Kaggle.")

Treinando o modelo Random Forest com todos os dados de treino...
Modelo Random Forest treinado com sucesso!
Fazendo as previsões com o Random Forest...

Arquivo 'submissao_random_forest.csv' foi criado com sucesso!
Ele está pronto para uma nova submissão no Kaggle.


## **Versão 3 (Feature de Título)**
- A engenharia da feature Title a partir do nome não alterou a pontuação da Regressão Logística, indicando que a informação talvez já estivesse implícita em outras features.

In [23]:

# Executando os dados brutos novamente
caminho_treino = '/content/drive/MyDrive/Kaggle_Titanic/train.csv'
caminho_teste = '/content/drive/MyDrive/Kaggle_Titanic/test.csv'

df_treino_novo = pd.read_csv(caminho_treino)
df_teste_novo = pd.read_csv(caminho_teste)

print("Dataframes originais recarregados com sucesso!")

Dataframes originais recarregados com sucesso!


In [24]:
# --- Bloco de Limpeza e Preparação (Versão Corrigida) ---

# 1. Tratar valores nulos (Idade, Embarque, Tarifa)
mediana_idade = df_treino_novo['Age'].median()
df_treino_novo['Age'].fillna(mediana_idade, inplace=True)
df_teste_novo['Age'].fillna(mediana_idade, inplace=True)

moda_embarked = df_treino_novo['Embarked'].mode()[0]
df_treino_novo['Embarked'].fillna(moda_embarked, inplace=True)

mediana_fare = df_treino_novo['Fare'].median()
df_teste_novo['Fare'].fillna(mediana_fare, inplace=True)

# 2. Remover colunas desnecessárias (AGORA MANTENDO 'Name')
df_treino_novo = df_treino_novo.drop(['Ticket', 'Cabin'], axis=1)
df_teste_novo = df_teste_novo.drop(['Ticket', 'Cabin'], axis=1)

# 3. Transformar colunas categóricas em números
df_treino_novo['Sex'] = df_treino_novo['Sex'].map({'male': 0, 'female': 1})
df_teste_novo['Sex'] = df_teste_novo['Sex'].map({'male': 0, 'female': 1})

df_treino_novo = pd.get_dummies(df_treino_novo, columns=['Embarked'], drop_first=True, dtype=int)
df_teste_novo = pd.get_dummies(df_teste_novo, columns=['Embarked'], drop_first=True, dtype=int)

print("Limpeza e transformação reaplicadas com sucesso!")
df_treino.info() # Verificando o resultado

Limpeza e transformação reaplicadas com sucesso!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked_Q   891 non-null    int64  
 9   Embarked_S   891 non-null    int64  
dtypes: float64(2), int64(8)
memory usage: 69.7 KB


/tmp/ipython-input-2146090009.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_treino_novo['Age'].fillna(mediana_idade, inplace=True)
/tmp/ipython-input-2146090009.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

In [25]:
# --- ETAPA 1: Engenharia de Atributos ---

# 1.1 Extrair o Título da coluna 'Name' em ambos os dataframes
# A expressão regular ' ([A-Za-z]+)\.' encontra uma palavra (composta de letras) que está entre um espaço e um ponto.
df_treino_novo['Title'] = df_treino_novo['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_teste_novo['Title'] = df_teste_novo['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# 1.2 Agrupar títulos raros ou sinônimos
# Criamos um loop para aplicar as mesmas regras nos dois dataframes
for df in [df_treino_novo, df_teste_novo]:
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

# 1.3 Mapear os títulos para números
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
df_treino_novo['Title'] = df_treino_novo['Title'].map(title_mapping)
df_teste_novo['Title'] = df_teste_novo['Title'].map(title_mapping)

# Se houver algum título no teste que não exista no treino, preenchemos com 0
df_teste_novo['Title'] = df_teste_novo['Title'].fillna(0)

# 1.4 Agora podemos remover a coluna 'Name', pois já usamos a informação que queríamos
df_treino_novo = df_treino_novo.drop(['Name'], axis=1)
df_teste_novo = df_teste_novo.drop(['Name'], axis=1)

print("Engenharia de atributos da coluna 'Title' concluída!")
print(df_treino_novo[['Title', 'Survived']].groupby(['Title']).mean()) # Ver a taxa de sobrevivência por título


# --- ETAPA 2: Treinar e Submeter com os Novos Dados ---

# 2.1 Preparar os novos dataframes X e y
X_novo = df_treino_novo.drop(['Survived', 'PassengerId'], axis=1)
y_novo = df_treino_novo['Survived']

# 2.2 Treinar nosso modelo campeão (Regressão Logística) com a NOVA feature 'Title'
modelo_final_com_titulo = LogisticRegression(max_iter=1000)
modelo_final_com_titulo.fit(X_novo, y_novo)

# 2.3 Fazer as novas previsões e criar o novo arquivo de submissão
previsoes_com_titulo = modelo_final_com_titulo.predict(df_teste_novo.drop('PassengerId', axis=1))

submissao_com_titulo = pd.DataFrame({
    "PassengerId": df_teste_novo["PassengerId"],
    "Survived": previsoes_com_titulo
})

# NOVO NOME DE ARQUIVO!
submissao_com_titulo.to_csv('submissao_com_titulos.csv', index=False)

print("\nArquivo 'submissao_com_titulos.csv' criado com sucesso!")

Engenharia de atributos da coluna 'Title' concluída!
       Survived
Title          
1      0.156673
2      0.702703
3      0.793651
4      0.575000
5      0.347826


<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-809040065.py:5: SyntaxWarning: invalid escape sequence '\.'
  df_treino_novo['Title'] = df_treino_novo['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
/tmp/ipython-input-809040065.py:6: SyntaxWarning: invalid escape sequence '\.'
  df_teste_novo['Title'] = df_teste_novo['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)



Arquivo 'submissao_com_titulos.csv' criado com sucesso!


## **Versão 4: Combinando o Melhor Modelo com a Melhor Feature**

In [26]:
# Estamos reutilizando os dataframes 'df_treino_novo' e 'df_teste_novo' que já têm a feature 'Title'.
# E os X_novo e y_novo que criamos a partir deles.

# 1. Treinar o modelo Random Forest com os dados que INCLUEM a feature 'Title'
print("Treinando o Random Forest com a feature 'Title'...")
modelo_rf_com_titulo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf_com_titulo.fit(X_novo, y_novo)
print("Modelo treinado com sucesso!")

# 2. Fazer as previsões com este novo modelo
previsoes_rf_com_titulo = modelo_rf_com_titulo.predict(df_teste_novo.drop('PassengerId', axis=1))

# 3. Criar o novo arquivo de submissão
submissao_rf_com_titulo = pd.DataFrame({
    "PassengerId": df_teste_novo["PassengerId"],
    "Survived": previsoes_rf_com_titulo
})

# NOVO NOME DE ARQUIVO!
submissao_rf_com_titulo.to_csv('submissao_rf_com_titulos.csv', index=False)

print("\nArquivo 'submissao_rf_com_titulos.csv' criado com sucesso!")

Treinando o Random Forest com a feature 'Title'...
Modelo treinado com sucesso!

Arquivo 'submissao_rf_com_titulos.csv' criado com sucesso!


## **Versão 5 (Features de Família)**
- A criação das features FamilySize e IsAlone resultou em um salto na pontuação para 0.77751! Isso provou o valor da engenharia de atributos.

In [27]:
# Partindo do df_treino_novo e df_teste_novo que já têm a coluna 'Title'

# 1. Criar a feature FamilySize
for df in [df_treino_novo, df_teste_novo]:
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# 2. Criar a feature IsAlone
for df in [df_treino_novo, df_teste_novo]:
    df['IsAlone'] = 0
    df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1

print("Novas features 'FamilySize' e 'IsAlone' criadas!")


# --- Treinando nosso modelo ESTÁVEL (Regressão Logística) com AINDA MAIS features ---

# Preparar os dados finais (agora com Title, FamilySize e IsAlone)
X_final = df_treino_novo.drop(['Survived', 'PassengerId'], axis=1)
y_final = df_treino_novo['Survived']

# Treinar o modelo de Regressão Logística
modelo_final_plus = LogisticRegression(max_iter=1000)
modelo_final_plus.fit(X_final, y_final)

# Fazer previsões
previsoes_final_plus = modelo_final_plus.predict(df_teste_novo.drop('PassengerId', axis=1))

# Criar arquivo de submissão
submissao_final_plus = pd.DataFrame({
    "PassengerId": df_teste_novo["PassengerId"],
    "Survived": previsoes_final_plus
})
submissao_final_plus.to_csv('submissao_com_familia.csv', index=False)

print("\nArquivo 'submissao_com_familia.csv' criado com sucesso!")

Novas features 'FamilySize' e 'IsAlone' criadas!

Arquivo 'submissao_com_familia.csv' criado com sucesso!


## **Versão 6 (Binning)**
- Agrupar Age e Fare em faixas (bins) melhorou ainda mais o modelo, alcançando nossa melhor pontuação: 0.77990.

In [28]:
# Partindo dos dataframes que já têm 'Title', 'FamilySize', e 'IsAlone'
df_treino_final = df_treino_novo.copy()
df_teste_final = df_teste_novo.copy()

# 1. Binning da Idade (Age)
# Criamos 4 faixas de idade
for df in [df_treino_final, df_teste_final]:
    df['Age_bin'] = pd.cut(df['Age'], bins=[0, 12, 20, 40, 120], labels=['Criança', 'Adolescente', 'Adulto', 'Idoso'])

# 2. Binning da Tarifa (Fare)
# Usamos qcut para dividir em 4 grupos com o mesmo número de pessoas
for df in [df_treino_final, df_teste_final]:
    df['Fare_bin'] = pd.qcut(df['Fare'], 4, labels=['Muito_Barato', 'Barato', 'Caro', 'Muito_Caro'])

# 3. Transformar as novas colunas categóricas em números
# Usamos get_dummies para as novas colunas
df_treino_final = pd.get_dummies(df_treino_final, columns=['Age_bin', 'Fare_bin'], drop_first=True, dtype=int)
df_teste_final = pd.get_dummies(df_teste_final, columns=['Age_bin', 'Fare_bin'], drop_first=True, dtype=int)

# 4. Remover as colunas originais de Age e Fare, pois já foram substituídas pelas faixas
df_treino_final = df_treino_final.drop(['Age', 'Fare'], axis=1)
df_teste_final = df_teste_final.drop(['Age', 'Fare'], axis=1)

print("Binning de Age e Fare concluído!")


# --- Treinando o modelo com os dados 'binnados' ---

# Preparar os dados finais
X_binned = df_treino_final.drop(['Survived', 'PassengerId'], axis=1)
y_binned = df_treino_final['Survived']

# Treinar o modelo de Regressão Logística
modelo_binned = LogisticRegression(max_iter=1000)
modelo_binned.fit(X_binned, y_binned)

# Fazer previsões
# Precisamos garantir que o df de teste tenha as mesmas colunas que o de treino
teste_cols = X_binned.columns
previsoes_binned = modelo_binned.predict(df_teste_final[teste_cols])

# Criar arquivo de submissão
submissao_binned = pd.DataFrame({
    "PassengerId": df_teste_final["PassengerId"],
    "Survived": previsoes_binned
})
submissao_binned.to_csv('submissao_com_bins.csv', index=False)

print("\nArquivo 'submissao_com_bins.csv' criado com sucesso!")

Binning de Age e Fare concluído!

Arquivo 'submissao_com_bins.csv' criado com sucesso!


## **Versão 7 (GridSearchCV)**


In [29]:
# Import Grid SearchCV
from sklearn.model_selection import GridSearchCV

# Usaremos os dados da nossa melhor versão até agora (com bins)
# X_binned e y_binned já devem estar no seu notebook

print("Iniciando a busca pelos melhores parâmetros para o Random Forest...")

# 1. Definir a grade de parâmetros que queremos testar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 8, 12],
    'min_samples_leaf': [3, 4, 5],
    'max_features': ['sqrt', 'log2']
}

# 2. Criar o objeto GridSearchCV
# cv=5 significa que ele usará validação cruzada com 5 partes para ter certeza do resultado
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1) # n_jobs=-1 usa todos os processadores para ir mais rápido

# 3. Executar a busca (esta é a parte que pode demorar)
grid_search.fit(X_binned, y_binned)

# 4. Imprimir os melhores parâmetros encontrados
print("\nMelhores parâmetros encontrados:")
print(grid_search.best_params_)

# 5. Usar o melhor modelo encontrado para fazer as previsões
best_rf_model = grid_search.best_estimator_

previsoes_finais_rf = best_rf_model.predict(df_teste_final[X_binned.columns])

# 6. Criar o arquivo de submissão final
submissao_final_rf = pd.DataFrame({
    "PassengerId": df_teste_final["PassengerId"],
    "Survived": previsoes_finais_rf
})
submissao_final_rf.to_csv('submissao_rf_otimizado.csv', index=False)

print("\nArquivo 'submissao_rf_otimizado.csv' criado com sucesso!")

Iniciando a busca pelos melhores parâmetros para o Random Forest...

Melhores parâmetros encontrados:
{'max_depth': 12, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'n_estimators': 300}

Arquivo 'submissao_rf_otimizado.csv' criado com sucesso!


## **Versão 8 (Mantendo os dados da cabine)**

In [30]:
# --- Bloco de Reset e Limpeza Novamente (Mantendo a Cabine) ---

# 1. Recarregar os dados originais
caminho_treino_nv = '/content/drive/MyDrive/Kaggle_Titanic/train.csv'
caminho_teste_nv = '/content/drive/MyDrive/Kaggle_Titanic/test.csv'
df_treino_nv = pd.read_csv(caminho_treino_nv)
df_teste_nv = pd.read_csv(caminho_teste_nv)

# 2. Preencher nulos de Age, Embarked e Fare (mas não de Cabin ainda)
mediana_idade = df_treino_nv['Age'].median()
df_treino_nv['Age'].fillna(mediana_idade, inplace=True)
df_teste_nv['Age'].fillna(mediana_idade, inplace=True)
moda_embarked = df_treino_nv['Embarked'].mode()[0]
df_treino_nv['Embarked'].fillna(moda_embarked, inplace=True)
mediana_fare = df_treino_nv['Fare'].median()
df_teste_nv['Fare'].fillna(mediana_fare, inplace=True)

print("Dados recarregados e nulos básicos preenchidos.")

Dados recarregados e nulos básicos preenchidos.


/tmp/ipython-input-4062139767.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_treino_nv['Age'].fillna(mediana_idade, inplace=True)
/tmp/ipython-input-4062139767.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

## **Versão 9 (Ensemble)**

In [31]:
# --- CÓDIGO FINAL: PREPARAÇÃO DO MELHOR DATASET E TREINAMENTO DO ENSEMBLE ---

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC

# --- Etapa 1: Preparar nosso melhor Dataset (o que gerou o score 0.77990) ---

# 1.1 Recarregar os dados originais para garantir um início limpo
caminho_treino = '/content/drive/MyDrive/Kaggle_Titanic/train.csv'
caminho_teste = '/content/drive/MyDrive/Kaggle_Titanic/test.csv'
df_treino = pd.read_csv(caminho_treino)
df_teste = pd.read_csv(caminho_teste)

# 1.2 Preencher nulos básicos
mediana_idade = df_treino['Age'].median()
df_treino['Age'].fillna(mediana_idade, inplace=True)
df_teste['Age'].fillna(mediana_idade, inplace=True)
moda_embarked = df_treino['Embarked'].mode()[0]
df_treino['Embarked'].fillna(moda_embarked, inplace=True)
mediana_fare = df_treino['Fare'].median()
df_teste['Fare'].fillna(mediana_fare, inplace=True)

# 1.3 Engenharia de atributos que funcionou (Title, FamilySize, Bins)
df_treino_fn = df_treino.copy()
df_teste_fn = df_teste.copy()

for df in [df_treino_fn, df_teste_fn]:
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = 1
    df.loc[df['FamilySize'] > 1, 'IsAlone'] = 0
    df['Age_bin'] = pd.cut(df['Age'], bins=[0, 12, 20, 40, 120], labels=['Criança', 'Adolescente', 'Adulto', 'Idoso'])
    df['Fare_bin'] = pd.qcut(df['Fare'], 4, labels=['Muito_Barato', 'Barato', 'Caro', 'Muito_Caro'], duplicates='drop')
    df['Fare_bin'].fillna(df['Fare_bin'].mode()[0], inplace=True)

# 1.4 Transformações numéricas
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
sex_mapping = {'male': 0, 'female': 1}
for df in [df_treino_fn, df_teste_fn]:
    df['Sex'] = df['Sex'].map(sex_mapping)
    df['Title'] = df['Title'].map(title_mapping).fillna(0)
colunas_para_dummies = ['Embarked', 'Age_bin', 'Fare_bin']
df_treino_fn = pd.get_dummies(df_treino_fn, columns=colunas_para_dummies, drop_first=True, dtype=int)
df_teste_fn = pd.get_dummies(df_teste_fn, columns=colunas_para_dummies, drop_first=True, dtype=int)

# 1.5 Limpeza final de colunas (aqui removemos 'Cabin' e as outras que não precisamos mais)
colunas_para_remover = ['Name', 'Ticket', 'Cabin', 'Age', 'Fare', 'SibSp', 'Parch']
df_treino_fn = df_treino_fn.drop(colunas_para_remover, axis=1)
df_teste_fn = df_teste_fn.drop(colunas_para_remover, axis=1)

print("Melhor dataset preparado com sucesso!")


# --- Etapa 2: Treinar o Comitê de Especialistas (Ensemble) ---

# 2.1 Preparar os dados para o treinamento
X_final = df_treino_fn.drop(['Survived', 'PassengerId'], axis=1)
y_final = df_treino_fn['Survived']
# Alinhar colunas
missing_cols = set(X_final.columns) - set(df_teste_fn.columns)
for c in missing_cols:
    df_teste_fn[c] = 0
df_teste_fn = df_teste_fn[X_final.columns]


# 2.2 Criar os 4 especialistas
clf1 = LogisticRegression(max_iter=2000, random_state=42)
clf2 = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_leaf=4, random_state=42) # RF 'domado'
clf3 = SVC(random_state=42)
clf4 = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)

# 2.3 Criar o Comitê de Votação
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('svc', clf3), ('gbc', clf4)], voting='hard')

# 2.4 Treinar o comitê
eclf1 = eclf1.fit(X_final, y_final)

# 2.5 Fazer previsões com base na votação da maioria
previsoes_ensemble = eclf1.predict(df_teste_fn)

# 2.6 Criar o arquivo de submissão
submissao_ensemble = pd.DataFrame({
    "PassengerId": df_teste['PassengerId'],
    "Survived": previsoes_ensemble
})
submissao_ensemble.to_csv('submissao_ensemble.csv', index=False)

print("\nArquivo 'submissao_ensemble.csv' criado com sucesso!")

Melhor dataset preparado com sucesso!


<>:30: SyntaxWarning: invalid escape sequence '\.'
<>:30: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-2490400090.py:30: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
/tmp/ipython-input-2490400090.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_treino['Age'].fillna(mediana_idade, inplace=True)
/tmp/ipython-input-2490400090.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment 


Arquivo 'submissao_ensemble.csv' criado com sucesso!


# **4. Conclusão Final do Projeto**
> Após um extenso processo de exploração, limpeza, engenharia de atributos e experimentação de modelos, o melhor resultado foi alcançado com um modelo de Regressão Logística alimentado por um conjunto de features cuidadosamente construídas (Title, FamilySize, IsAlone, e bins de Age/Fare).

- **Melhor Versão do challenge: Versão 06 (submissao_com_bins)

- **Melhor Pontuação no Kaggle: 0.77990**

- **Posição no Ranking: #3058 (Top ~20%)**

> **Obs.:** A principal lição deste projeto é que a qualidade da preparação dos dados e a inteligência na criação de features são frequentemente mais impactantes do que a complexidade do algoritmo.

O projeto serviu como uma jornada completa pelo pipeline de ciência de dados, resultando em um modelo competitivo e um grande aprendizado.

# Loading....